<a href="https://colab.research.google.com/github/azibit/pycool/blob/master/PyCool_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
DATASET_DOWNLOAD_LINK = "https://aidris559lab4.s3.amazonaws.com/Cassava_Disease_Dataset_for_iCassava/Balanced/Balanced_with_white_background_NS.zip"
FILE_TO_UNZIP = DATASET_DOWNLOAD_LINK.split('/')[-1]
!wget -c {DATASET_DOWNLOAD_LINK}
!unzip {FILE_TO_UNZIP}

Streaming output truncated to the last 5000 lines.
  inflating: Balanced_with_white_background_NS/test/cgm/test-cgm-143.jpg  
  inflating: Balanced_with_white_background_NS/test/cgm/test-cgm-144.jpg  
  inflating: Balanced_with_white_background_NS/test/cgm/test-cgm-145.jpg  
  inflating: Balanced_with_white_background_NS/test/cgm/test-cgm-146.jpg  
  inflating: Balanced_with_white_background_NS/test/cgm/test-cgm-147.jpg  
  inflating: Balanced_with_white_background_NS/test/cgm/test-cgm-148.jpg  
  inflating: Balanced_with_white_background_NS/test/cgm/test-cgm-149.jpg  
  inflating: Balanced_with_white_background_NS/test/cgm/test-cgm-15.jpg  
  inflating: Balanced_with_white_background_NS/test/cgm/test-cgm-150.jpg  
  inflating: Balanced_with_white_background_NS/test/cgm/test-cgm-151.jpg  
  inflating: Balanced_with_white_background_NS/test/cgm/test-cgm-152.jpg  
  inflating: Balanced_with_white_background_NS/test/cgm/test-cgm-153.jpg  
  inflating: Balanced_with_white_background_NS/tes

In [31]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import time
import os
import copy
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
import random

plt.ion()   # interactive mode

In [48]:
class TrainModel():
  def __init__(self, dataset_dir, batch_size, img_size, channels):
    self.dataset_dir = dataset_dir
    self.batch_size = batch_size

    self.img_size = img_size
    self.channels = channels

    

  def compute_mean_and_std(self):
    data_transforms_wo_normalization = {
      'train': transforms.Compose([
          transforms.RandomResizedCrop(self.img_size),
          transforms.RandomHorizontalFlip(),
          transforms.ToTensor(),
      ])
    }

    print("Computing Mean and STD")
    train_dataset = datasets.ImageFolder(os.path.join(self.dataset_dir, 'train'),
                                              data_transforms_wo_normalization['train'])

    train_dataloader = DataLoader(train_dataset, batch_size=self.batch_size,
                                                shuffle=True)

    train_dataloader = DataLoader(train_dataset, batch_size=len(train_dataset), shuffle=True)
    data = next(iter(train_dataloader))

    custom_mean = []
    custom_std = []

    
    for channels in range(self.channels):
      custom_mean.append(data[0][:, channels, :, :].mean())
      custom_std.append(data[0][:, channels, :, :].std())
      print("...", end='')

    self.mean = custom_mean
    self.std = custom_std

  def load_dataset(self):
    self.compute_mean_and_std()
    data_transforms_with_normalization = {
        'train': transforms.Compose([
            transforms.RandomResizedCrop(self.img_size),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(self.mean, self.std)
        ]),
        'val': transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(self.img_size),
            transforms.ToTensor(),
            transforms.Normalize(self.mean, self.std)
        ]),
    }

    train_dataset = datasets.ImageFolder(os.path.join(self.dataset_dir, 'train'),
                                            data_transforms_with_normalization['train'])

    train_dataloader = DataLoader(train_dataset, batch_size=self.batch_size,
                                                shuffle=True)

    val_dataset = datasets.ImageFolder(os.path.join(self.dataset_dir, 'val'),
                                              data_transforms_with_normalization['val'])

    val_dataloader = DataLoader(val_dataset, batch_size=self.batch_size,
                                                shuffle=True)
    
    dataloader = {'train': train_dataloader,
                'val': val_dataloader}

    dataset_sizes = {'train': len(train_dataset),
                  'val': len(val_dataset)}

    classnames = {'train': train_dataset.classes,
                'val': val_dataset.classes}

    self.dataloader = dataloader 
    self.dataset_sizes = dataset_sizes
    self.classnames = classnames

  def set_model_to_use(self, model):
    self.model = model
    self.build_transfer_learning_model(self.device)

  def set_device(self, device):
    self.device =  device

  def build_transfer_learning_model(self, device):
    # model_ft = models.densenet121(pretrained=True) # Choose a model to use
    # model_ft = self.model
    num_ftrs = self.model.classifier.in_features
    self.model.fc = nn.Linear(num_ftrs, len(self.classnames['train']))

    self.model = self.model.to(device)

    self.criterion = nn.CrossEntropyLoss()

    # Observe that all parameters are being optimized
    self.optimizer = optim.SGD(self.model.parameters(), lr=0.001, momentum=0.9)

    # Decay LR by a factor of 0.1 every 7 epochs
    self.scheduler = lr_scheduler.StepLR(self.optimizer, step_size=7, gamma=0.1)

  def train_model(self, num_epochs=25):
    since = time.time() # To measure time taken to build and train the model

    best_model_wts = copy.deepcopy(self.model.state_dict()) # Save weight of best model
    best_acc = 0.0 # Save best accuracy

    self.training_loss = []
    self.val_loss = []

    self.training_accuracy = []
    self.val_accuracy = []

    for epoch in range(num_epochs):
      print('Epoch {}/{}'.format(epoch, num_epochs - 1))
      print('-' * 10)

      # Each epoch has its training and validation
      for phase in ['train', 'val']:
        if phase == 'train':
          self.model.train()
        else:
          self.model.eval()

        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in self.dataloader[phase]:
          inputs = inputs.to(self.device) # Move data to GPU if you are using GPU
          labels = labels.to(self.device) # Move data to GPU if you are using GPU

          self.optimizer.zero_grad() # Set gradients to zero so it does not accumulate

          with torch.set_grad_enabled(phase == 'train'): # Train in train phase
            outputs = self.model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = self.criterion(outputs, labels) # Compare ground truth with outputs

            # Backward propagation if in train mode
            if phase == 'train':
              loss.backward()
              self.optimizer.step()

          # Compute loss and accuracy
          running_loss += loss.item() * inputs.size(0)
          
          
          running_corrects += torch.sum(preds == labels.data)

        if phase =='train':
          self.scheduler.step()

        
        epoch_loss = running_loss / self.dataset_sizes[phase]
        epoch_acc = running_corrects.double() / self.dataset_sizes[phase]

        if phase == 'train':
          self.training_loss.append(epoch_loss)
          self.training_accuracy.append(epoch_acc)
        else:
          self.val_loss.append(epoch_loss)
          self.val_accuracy.append(epoch_acc)

        print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

        # deep copy the model
        if phase == 'val' and epoch_acc > best_acc:
          best_acc = epoch_acc
          best_model_wts = copy.deepcopy(self.model.state_dict())

      print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    self.model.load_state_dict(best_model_wts) # Load the best model weights

  def plot_loss_graph():
    fig = plt.figure()
    plt.plot(self.training_loss, color='blue')
    plt.plot(self.val_loss, color='red')
    plt.legend(['Train Loss', 'Val Loss'], loc='upper right')
    plt.xlabel('Number of epoch completed')
    plt.ylabel('Loss')

  def plot_accuracy_graph():
    fig = plt.figure()
    plt.plot(self.training_accuracy, color='blue')
    plt.plot(self.val_accuracy, color='red')
    plt.legend(['Train Accuracy', 'Val Accuracy'], loc='upper left')
    plt.xlabel('Number of epoch completed')
    plt.ylabel('Accuracy')

In [49]:
DATA_DIR = FILE_TO_UNZIP.split('.')[0]
train_model = TrainModel(DATA_DIR, 64, 224, 3)
train_model.load_dataset()

Computing Mean and STD
.........

In [50]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
train_model.set_device(device)

model_ft = models.densenet121(pretrained=True)
train_model.set_model_to_use(model_ft)

In [ ]:
train_model.train_model()

Epoch 0/24
----------
train Loss: 4.3223 Acc: 0.2896
val Loss: 1.0569 Acc: 0.6231

Epoch 1/24
----------
train Loss: 0.8971 Acc: 0.6651
val Loss: 0.8963 Acc: 0.6797

Epoch 2/24
----------
train Loss: 0.7111 Acc: 0.7389
val Loss: 0.7725 Acc: 0.7110

Epoch 3/24
----------
train Loss: 0.6146 Acc: 0.7869
val Loss: 0.6747 Acc: 0.7650

Epoch 4/24
----------
train Loss: 0.5126 Acc: 0.8238
val Loss: 0.6856 Acc: 0.7676

Epoch 5/24
----------
train Loss: 0.4622 Acc: 0.8349
val Loss: 0.6958 Acc: 0.7575

Epoch 6/24
----------
train Loss: 0.4195 Acc: 0.8460
val Loss: 0.6188 Acc: 0.7888

Epoch 7/24
----------
train Loss: 0.3931 Acc: 0.8623
val Loss: 0.6153 Acc: 0.7877

Epoch 8/24
----------


In [ ]:
def plot_accuracy_graph(training_accuracy, val_accuracy):
  fig = plt.figure()
  plt.plot(training_accuracy, color='blue')
  plt.plot(val_accuracy, color='red')
  plt.legend(['Train Accuracy', 'Val Accuracy'], loc='upper left')
  plt.xlabel('Number of epoch completed')
  plt.ylabel('Accuracy')

In [ ]:
def gradcam_result(img_mean, img_std, pil_opened_image, model, model_name, the_layer_name, image_size = (224, 224)):
  normalizer = Normalize(mean=img_mean, std=img_std)
  torch_img = torch.from_numpy(np.asarray(pil_opened_image)).permute(2, 0, 1).unsqueeze(0).float().div(255).cuda()
  torch_img = F.upsample(torch_img, size=image_size, mode='bilinear', align_corners=False)
  normed_torch_img = normalizer(torch_img)

  model.eval(), model.cuda();
  model_dict = dict(type=model_name, arch=model, layer_name=the_layer_name, input_size=image_size)

  gradcam = GradCAM(model_dict, True)
  gradcampp = GradCAMpp(model_dict, True)

  mask, _ = gradcam(normed_torch_img)
  heatmap, result = visualize_cam(mask.cpu(), torch_img)

  mask_pp, _ = gradcam_pp(normed_torch_img)
  heatmap_pp, result_pp = visualize_cam(mask_pp.cpu(), torch_img)

  return torch.stack([torch_img.squeeze().cpu(), heatmap, heatmap_pp, result, result_pp], 0)

In [ ]:
images = gradcam_result()

images = make_grid(torch.cat(images, 0), nrow=5)